#####The UI used for this application is taken from Argon Design System

In [ ]:
!sudo apt -qq install ffmpeg
!pip install git+https://github.com/openai/whisper.git -qq
!pip install pydub -qq
!pip install flask -qq
!pip install flask_cors -qq
!pip install flask-ngrok -qq
!pip install pyngrok==4.1.1 -qq
!pip install yt_dlp -qq
!pip install soundfile -qq

ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 87.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 20.0 MB/s eta 0:00:00
     ━━

In [ ]:
!git clone https://github.com/coqui-ai/TTS.git
%cd "/content/TTS"
! apt-get -qq install espeak
! pip install -q -e .

Cloning into 'TTS'...
remote: Enumerating objects: 32550, done.
remote: Counting objects: 100% (5181/5181), done.
remote: Compressing objects: 100% (854/854), done.
remote: Total 32550 (delta 4697), reused 4467 (delta 4297), pack-reused 27369
Receiving objects: 100% (32550/32550), 166.13 MiB | 15.11 MiB/s, done.
Resolving deltas: 100% (23615/23615), done.
/content/TTS
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 120882 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1.1_amd64.deb ...
Unpacking libportaudio2:amd64 (19.6.0-1.1) ...
Selecting previously unselected package libsonic0:amd64.
Preparing to unpack .../libsonic0_0.2.0-11build1_amd64.deb ...
Unpacking libsonic0:amd64 (0.2.0-11build1) ...
Selecting previously unselected package espeak-data:amd64.
Preparing to unpack .../espeak-data_1.48.15+dfsg-3_amd64.deb ...
Unpacking espeak-data:amd64 (1.48.15+dfsg-3) ...
Selecting previously unselected package libes

In [ ]:
OUTPUT_VIDEO = "/content/DownloadedVideos/downloadedVideo.mkv"
OUTPUT_AUDIO = "/content/OutcomeAudios/extractedAudio.wav"
AUDIO_FOLDER = "/content/OutcomeAudios/"
VIDEO_FOLDER = "/content/DownloadedVideos/"
SAMPLING_RATE = 28000

In [ ]:
import whisper
import json
modelWhisper = whisper.load_model("medium")

100%|██████████████████████████████████████| 1.42G/1.42G [00:13<00:00, 114MiB/s]


In [ ]:
import torch
from TTS.api import TTS

# Get device
device = "cuda" if torch.cuda.is_available() else "cpu"
# Init TTS
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)

 > You must agree to the terms of service to use this model.
 | > Please see the terms of service at https://coqui.ai/cpml.txt
 | > "I have read, understood and agreed to the Terms and Conditions." - [y/n]
 | | > y
 > Downloading model to /root/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v2


100%|█████████▉| 1.86G/1.87G [01:11<00:00, 27.7MiB/s]
100%|██████████| 1.87G/1.87G [01:11<00:00, 26.0MiB/s]
100%|██████████| 4.37k/4.37k [00:00<00:00, 9.75kiB/s]

100%|██████████| 361k/361k [00:00<00:00, 371kiB/s]


 > Model's license - CPML
 > Check https://coqui.ai/cpml.txt for more info.
 > Using model: xtts


In [ ]:
def convert(speech):
  tts.tts_to_file(text=speech, speaker_wav="/content/OutcomeAudios/extractedAudio.wav", language="en", file_path="/content/OutcomeAudios/translatedAudio.wav")
  file_path = "/content/OutcomeAudios/translatedAudio.wav"
  return file_path

In [ ]:
#from bark import generate_audio, preload_models
from scipy.io.wavfile import write as write_wav
from IPython.display import Audio
import scipy
# download and load all models
# preload_models()

In [ ]:
import os

def checkAndMakeDir(folder):
  if os.path.exists(folder)==False:
    os.makedirs(folder)

checkAndMakeDir(AUDIO_FOLDER)
checkAndMakeDir(VIDEO_FOLDER)

In [ ]:
from yt_dlp import YoutubeDL
from scipy.io.wavfile import read as read_wav

def downloadVideoYT(URL):
  if os.path.isfile(OUTPUT_VIDEO):
      os.remove(OUTPUT_VIDEO)
  if os.path.isfile(OUTPUT_AUDIO):
      os.remove(OUTPUT_AUDIO)
  ytdl_format_options = {
    'outtmpl': OUTPUT_VIDEO,
    'format': 'best[ext=mp4]'
  }
  with YoutubeDL(ytdl_format_options) as ydl:
    ydl.download([URL])
  print("Download Complete")

def extractAudio():
  import subprocess
  subprocess.call(["ffmpeg",
                   "-y",
                   "-i",
                   OUTPUT_VIDEO,
                   OUTPUT_AUDIO])
  SAMPLING_RATE, data=read_wav("/content/OutcomeAudios/extractedAudio.wav")
  print("Audio Extraction Complete")

In [ ]:
def transcription():
  translatedText = modelWhisper.transcribe("/content/OutcomeAudios/extractedAudio.wav", task = 'translate')
  print(translatedText)
  # json_object = json.dumps(translatedText, indent=4)
  # with open("/content/outputFile.json", "w") as outfile:
  #     outfile.write(json_object)
  return translatedText

In [ ]:
def convertToArray():
  f = open("/content/outputFile.json")
  data = json.load(f)
  sentences = []
  data2 = []

  for segment in data['segments']:
    start_time = segment['start']
    end_time = segment['end']
    text = segment['text'].strip()
    sentences.append(text)
    newDict = dict(start_time = start_time, end_time = end_time, text = text)
    data2.append(newDict)

  newFile = json.dumps(data2, indent=4)
  with open("/content/outputFileMain.json", "w") as fileOut:
    fileOut.write(newFile)

  return sentences

In [ ]:
# import soundfile as sf
# import numpy as np
# import nltk
# nltk.download('punkt')
# from bark.api import semantic_to_waveform
# from bark.generation import generate_text_semantic

# def convert(speech):
#   sentences = nltk.sent_tokenize(speech)
#   SPEAKER = "v2/en_speaker_3"
#   GEN_TEMP = 0.8
#   silence = np.zeros(int(0.25 * SAMPLING_RATE))  # quarter second of silence
#   pieces = []
#   for sentence in sentences:
#     semantic_tokens = generate_text_semantic(
#         sentence, #Tokenized sentence
#         history_prompt=SPEAKER, #speaker model
#         temp=GEN_TEMP, #Threshold for randomness in generated voice
#         min_eos_p=0.05,  # this controls how likely the generation is to end
#     )
#     audio_array = semantic_to_waveform(semantic_tokens, history_prompt=SPEAKER,)
#     pieces += [audio_array, silence.copy()]

#   final_piece=np.concatenate(pieces)
#   sf.write("output_audio.wav", final_piece, SAMPLING_RATE)
#   # Get the full path to the saved file
#   file_path = os.path.abspath("output_audio.wav")
#   return file_path

In [ ]:
from pydub import AudioSegment, silence
from pydub.playback import play

def createFinalAudio(audioSegment):
  myaudio = intro = AudioSegment.from_wav("/content/OutcomeAudios/extractedAudio.wav")
  dBFS=myaudio.dBFS
  silenceOut = silence.detect_silence(myaudio, min_silence_len=1000, silence_thresh=dBFS-16)

  finalAudio = AudioSegment.silent(duration = silenceOut[0][1])
  song = AudioSegment.from_wav(audioSegment)
  finalAudio += song

  finalFilePath = "/content/OutcomeAudios/translatedAudioWithSilence.wav"
  finalAudio.export(finalFilePath, format="wav")
  return finalFilePath

In [ ]:
import subprocess

def addAudioBackToVideo(audio_path, video_path, output_video_path):
  subprocess.run(f"ffmpeg -i {video_path} -i {audio_path} -c:v copy -map 0:v:0 -map 1:a:0 {output_video_path}", shell=True)

In [ ]:
# VIDEO = "https://www.youtube.com/watch?v=xAEyTzNfqnM"
# downloadVideoYT(VIDEO)
# extractAudio()
# print(SAMPLING_RATE)
# translatedText = transcription()
# print(translatedText)
# print("TRANSCRIPTION COMPLETE!")
# translatedArray = convertToArray()
# audioSegment = convert(translatedText)
# print("GENERATING AUDIO COMPLETE!")
# finalAudioPath = createFinalAudio(audioSegment)
# print(finalAudioPath)
# addAudioBackToVideo(audio_path="/content/FinalAudio.wav",
#                      video_path="/content/DownloadedVideos/downloadedVideo.mkv",
#                      output_video_path="/content/static/translatedVideoFinal.mkv")
# print("COMPLETE!")

In [ ]:
!ngrok authtoken <NGROCK AUTH TOKEN>
# See https://dashboard.ngrok.com/get-started/setup to generate the Auth Token

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
checkAndMakeDir('/content/templates')
checkAndMakeDir('/content/static')

In [ ]:
from google.colab import files
# Upload Assets.zip
%cd /content/
uploaded = files.upload()
%cd /content/static/
!unzip /content/assets.zip
%cd /content/

In [ ]:
#https://www.youtube.com/shorts/6f-_j_BSUwY

from flask_ngrok import run_with_ngrok
from flask import Flask, redirect, url_for, render_template, request, Response
from google.colab import files

import os
import shutil

app = Flask(__name__)
run_with_ngrok(app)

if not os.path.isfile('/content/static/styles.css'):
  %cd /content/static/
  uploaded = files.upload()
if not os.path.isfile('/content/templates/index.html'):
  %cd /content/templates/
  uploaded = files.upload()
%cd /content/

OUTPUT_VIDEO = "/content/DownloadedVideos/downloadedVideo.mkv"
OUTPUT_AUDIO = "/content/OutcomeAudios/extractedAudio.wav"
AUDIO_FOLDER = "/content/OutcomeAudios/"
VIDEO_FOLDER = "/content/DownloadedVideos/"


@app.route("/")
def home():
    return render_template('index.html')

@app.route("/about")
def about():
    return render_template('about.html')

@app.route("/how-it-works")
def howItWorks():
    return render_template('howItWorks.html')

@app.route("/translate")
def translate():
    return render_template('code.html', showVideos = False)

@app.route("/admin")
def admin():
    return redirect(url_for("home"))

@app.route("/translate_video", methods=["GET", "POST"])
def translate_video():
    isURL = False

    if os.path.isfile(OUTPUT_VIDEO):
        os.remove(OUTPUT_VIDEO)
    if os.path.isfile(OUTPUT_AUDIO):
        os.remove(OUTPUT_AUDIO)
    if os.path.isfile("/content/OutcomeAudios/translatedAudio.wav"):
        os.remove("/content/OutcomeAudios/translatedAudio.wav")
    if os.path.isfile('/content/static/translatedVideoFinal.mkv'):
        os.remove("/content/static/translatedVideoFinal.mkv")

    video_url = request.form.get('videoUrl')
    video_file = request.files.get('videoFile')
    finalAudioPath = ""
    if video_url:
        isURL = True
        downloadVideoYT(video_url)
        extractAudio()
    elif video_file:
        filename = video_file.filename
        file_path = OUTPUT_VIDEO
        video_file.save(file_path)
        extractAudio()

    translatedText = transcription()
    audioSegment = convert(translatedText['text'])
    #finalAudioPath = createFinalAudio(audioSegment, translatedText['segments'])
    shutil.copyfile("/content/DownloadedVideos/downloadedVideo.mkv", '/content/static/origninalVideo.mkv')
    addAudioBackToVideo(audio_path= audioSegment,
                     video_path="/content/DownloadedVideos/downloadedVideo.mkv",
                     output_video_path="/content/static/translatedVideoFinal.mkv")

    return render_template('code.html', showVideos=True)


if __name__ == "__main__":
    app.run()

/content
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://aa6f-34-87-72-154.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [04/Dec/2023 07:20:51] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Dec/2023 07:20:52] "GET /static/assets/css/nucleo-icons.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Dec/2023 07:20:52] "GET /static/assets/css/nucleo-svg.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Dec/2023 07:20:52] "GET /static/assets/css/font-awesome.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Dec/2023 07:20:52] "GET /static/assets/css/argon-design-system.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Dec/2023 07:20:52] "GET /static/assets/img/brand/blue.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Dec/2023 07:20:52] "GET /static/assets/img/brand/white.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Dec/2023 07:20:58] "GET /static/assets/fonts/fontawesome-webfont.woff2?v=4.7.0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Dec/2023 07:20:58] "GET /static/assets/fonts/nucleo-icons.woff2 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Dec/2

[youtube] Extracting URL: https://www.youtube.com/shorts/6f-_j_BSUwY
[youtube] 6f-_j_BSUwY: Downloading webpage
[youtube] 6f-_j_BSUwY: Downloading ios player API JSON
[youtube] 6f-_j_BSUwY: Downloading android player API JSON
[youtube] 6f-_j_BSUwY: Downloading m3u8 information
[info] 6f-_j_BSUwY: Downloading 1 format(s): 22
[download] Destination: /content/DownloadedVideos/downloadedVideo.mkv
[download] 100% of    4.46MiB in 00:00:00 at 11.81MiB/s  
Download Complete
Audio Extraction Complete
{'text': " Last question, do you speak Japanese? I can speak a little. How did you learn Japanese? I went to a Japanese language school. What's the difference between someone who can speak Japanese and someone who can't? You go to school and what do you do after that? You go to school and spend 4 hours with someone who can speak English. Don't play with that person. Sorry, I meant in English. Force yourself to hang out with Japanese people. You put yourself in the situation that you need to speak 

INFO:werkzeug:127.0.0.1 - - [04/Dec/2023 07:21:37] "POST /translate_video HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Dec/2023 07:21:37] "GET /static/assets/css/nucleo-icons.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [04/Dec/2023 07:21:37] "GET /static/assets/css/nucleo-svg.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [04/Dec/2023 07:21:37] "GET /static/assets/css/argon-design-system.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [04/Dec/2023 07:21:37] "GET /static/assets/css/font-awesome.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [04/Dec/2023 07:21:38] "GET /static/origninalVideo.mkv HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [04/Dec/2023 07:21:38] "GET /static/assets/img/brand/blue.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [04/Dec/2023 07:21:38] "GET /static/translatedVideoFinal.mkv HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [04/Dec/2023 07:21:55] "GET /static/origninalVideo.mkv HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [04/Dec/2023 07:21:55] "GET /static/

In [ ]:
#audioSegment = convert(translatedText)
#finalAudioPath = createFinalAudio(audioSegment)